In [8]:
!git status


fatal: not a git repository (or any of the parent directories): .git


In [ ]:

"""
SCRIPT COMPLETO: Extracción de Datos Agrícolas para Videojuego
Región: Cusco, Perú
Datos: NDVI, Temperatura, Precipitación
Autor: Para Hackathon FarmGuardians
"""

# ============================================
# PASO 1: INSTALACIÓN Y CONFIGURACIÓN
# ============================================
# Ejecutar en Colab:
# !pip install earthengine-api geemap pandas numpy matplotlib

import ee
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

# ============================================
# PASO 2: AUTENTICACIÓN
# ============================================
# Primera vez: ee.Authenticate()
ee.Initialize(project='tu-proyecto-aqui')  # Cambiar por tu proyecto

# ============================================
# PASO 3: DEFINIR ÁREA DE CUSCO
# ============================================

# Opción A: Punto específico (Plaza de Armas, Cusco)
cusco_point = ee.Geometry.Point([-71.9675, -13.5319])

# Opción B: Región completa de Cusco
cusco_region = ee.Geometry.Polygon([
    [-72.5, -13.0],
    [-71.0, -13.0],
    [-71.0, -14.5],
    [-72.5, -14.5],
    [-72.5, -13.0]
])

# Opción C: Valle Sagrado (zona agrícola importante)
valle_sagrado = ee.Geometry.Polygon([
    [-72.2, -13.2],
    [-71.8, -13.2],
    [-71.8, -13.4],
    [-72.2, -13.4],
    [-72.2, -13.2]
])

# Elegir geometría para análisis
area_estudio = cusco_point  # Cambiar según necesites

# ============================================
# PASO 4: FUNCIONES DE EXTRACCIÓN
# ============================================

def extraer_ndvi_sentinel2(geometry, fecha_inicio, fecha_fin):
    """
    Extrae NDVI de Sentinel-2 (10m resolución)
    Ideal para parcelas específicas
    """
    def calcular_ndvi(image):
        ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
        return ndvi.copyProperties(image, ['system:time_start'])

    collection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
        .filterBounds(geometry) \
        .filterDate(fecha_inicio, fecha_fin) \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30)) \
        .map(calcular_ndvi)

    return collection


def extraer_ndvi_modis(geometry, fecha_inicio, fecha_fin):
    """
    Extrae NDVI de MODIS (250m resolución)
    Mejor cobertura temporal, datos desde 2000
    """
    collection = ee.ImageCollection('MODIS/061/MOD13Q1') \
        .filterDate(fecha_inicio, fecha_fin) \
        .select('NDVI')

    def escalar_ndvi(image):
        return image.multiply(0.0001).copyProperties(image, ['system:time_start'])

    return collection.map(escalar_ndvi)


def extraer_temperatura(geometry, fecha_inicio, fecha_fin):
    """
    Extrae temperatura superficial (°C)
    """
    collection = ee.ImageCollection('MODIS/061/MOD11A2') \
        .filterDate(fecha_inicio, fecha_fin) \
        .select('LST_Day_1km')

    def convertir_celsius(image):
        celsius = image.multiply(0.02).subtract(273.15)
        return celsius.copyProperties(image, ['system:time_start'])

    return collection.map(convertir_celsius)


def extraer_precipitacion(geometry, fecha_inicio, fecha_fin):
    """
    Extrae precipitación diaria (mm)
    """
    collection = ee.ImageCollection('UCSB-CHG/CHIRPS/DAILY') \
        .filterDate(fecha_inicio, fecha_fin) \
        .select('precipitation')

    return collection


def collection_a_dataframe(image_collection, geometry, escala, nombre_banda):
    """
    Convierte ImageCollection a DataFrame de pandas
    """
    def extraer_valor(image):
        valor = image.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=geometry,
            scale=escala,
            maxPixels=1e9
        ).get(nombre_banda)

        fecha = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd')

        return ee.Feature(None, {
            'fecha': fecha,
            nombre_banda: valor
        })

    features = image_collection.map(extraer_valor)
    info = features.getInfo()

    datos = [f['properties'] for f in info['features']]
    df = pd.DataFrame(datos)
    df['fecha'] = pd.to_datetime(df['fecha'])

    return df


# ============================================
# PASO 5: EXTRAER DATOS (2023-2024)
# ============================================

print("🌱 Extrayendo datos de Cusco para videojuego...")

fecha_inicio = '2023-01-01'
fecha_fin = '2024-12-31'

# NDVI (usa Sentinel-2 para alta resolución)
print("📊 Extrayendo NDVI...")
ndvi_collection = extraer_ndvi_sentinel2(area_estudio, fecha_inicio, fecha_fin)
ndvi_df = collection_a_dataframe(ndvi_collection, area_estudio, 10, 'NDVI')

# Temperatura
print("🌡️ Extrayendo temperatura...")
temp_collection = extraer_temperatura(area_estudio, fecha_inicio, fecha_fin)
temp_df = collection_a_dataframe(temp_collection, area_estudio, 1000, 'LST_Day_1km')
temp_df.rename(columns={'LST_Day_1km': 'temperatura_celsius'}, inplace=True)

# Precipitación
print("💧 Extrayendo precipitación...")
precip_collection = extraer_precipitacion(area_estudio, fecha_inicio, fecha_fin)
precip_df = collection_a_dataframe(precip_collection, area_estudio, 5000, 'precipitation')
precip_df.rename(columns={'precipitation': 'precipitacion_mm'}, inplace=True)

print("✅ Datos extraídos exitosamente!")

# ============================================
# PASO 6: COMBINAR Y LIMPIAR DATOS
# ============================================

# Unir todos los dataframes
datos_completos = ndvi_df.merge(temp_df, on='fecha', how='outer') \
                         .merge(precip_df, on='fecha', how='outer')

# Ordenar por fecha
datos_completos = datos_completos.sort_values('fecha').reset_index(drop=True)

# Interpolar valores faltantes
datos_completos['NDVI'] = datos_completos['NDVI'].interpolate(method='linear')
datos_completos['temperatura_celsius'] = datos_completos['temperatura_celsius'].interpolate(method='linear')
datos_completos['precipitacion_mm'] = datos_completos['precipitacion_mm'].fillna(0)

# ============================================
# PASO 7: CREAR MÉTRICAS PARA EL JUEGO
# ============================================

def calcular_metricas_juego(df):
    """
    Crea métricas útiles para mecánicas de videojuego
    """
    # Salud del cultivo (0-100)
    df['salud_cultivo'] = (df['NDVI'] * 100).clip(0, 100)

    # Estrés por calor
    df['estres_calor'] = pd.cut(
        df['temperatura_celsius'],
        bins=[-np.inf, 15, 25, 35, np.inf],
        labels=['Frío', 'Óptimo', 'Caluroso', 'Extremo']
    )

    # Necesidad de riego
    df['necesidad_riego'] = pd.cut(
        df['precipitacion_mm'],
        bins=[-np.inf, 5, 15, 30, np.inf],
        labels=['Crítico', 'Alto', 'Moderado', 'Bajo']
    )

    # Índice de productividad (combinado)
    df['productividad'] = (
        df['salud_cultivo'] * 0.5 +
        (100 - abs(df['temperatura_celsius'] - 20) * 2).clip(0, 100) * 0.3 +
        df['precipitacion_mm'].clip(0, 50) * 0.4
    ).clip(0, 100)

    # Estación del año (hemisferio sur)
    df['mes'] = df['fecha'].dt.month
    estaciones = {
        12: 'Verano', 1: 'Verano', 2: 'Verano',
        3: 'Otoño', 4: 'Otoño', 5: 'Otoño',
        6: 'Invierno', 7: 'Invierno', 8: 'Invierno',
        9: 'Primavera', 10: 'Primavera', 11: 'Primavera'
    }
    df['estacion'] = df['mes'].map(estaciones)

    return df

datos_juego = calcular_metricas_juego(datos_completos)

# ============================================
# PASO 8: EXPORTAR DATOS
# ============================================

# CSV para análisis
datos_juego.to_csv('cusco_datos_agricolas_juego.csv', index=False)
print("💾 Datos guardados en: cusco_datos_agricolas_juego.csv")

# JSON para Unity/Godot/Unreal
datos_json = datos_juego.to_dict(orient='records')
import json
with open('cusco_datos_agricolas_juego.json', 'w', encoding='utf-8') as f:
    json.dump(datos_json, f, indent=2, default=str, ensure_ascii=False)
print("💾 Datos guardados en: cusco_datos_agricolas_juego.json")

# ============================================
# PASO 9: VISUALIZACIÓN
# ============================================

fig, axes = plt.subplots(4, 1, figsize=(15, 12))

# NDVI
axes[0].plot(datos_juego['fecha'], datos_juego['NDVI'], color='green', linewidth=2)
axes[0].set_title('NDVI - Salud de la Vegetación en Cusco', fontsize=14, fontweight='bold')
axes[0].set_ylabel('NDVI')
axes[0].grid(True, alpha=0.3)

# Temperatura
axes[1].plot(datos_juego['fecha'], datos_juego['temperatura_celsius'], color='red', linewidth=2)
axes[1].set_title('Temperatura Superficial (°C)', fontsize=14, fontweight='bold')
axes[1].set_ylabel('°C')
axes[1].grid(True, alpha=0.3)

# Precipitación
axes[2].bar(datos_juego['fecha'], datos_juego['precipitacion_mm'], color='blue', alpha=0.6)
axes[2].set_title('Precipitación Diaria (mm)', fontsize=14, fontweight='bold')
axes[2].set_ylabel('mm')
axes[2].grid(True, alpha=0.3)

# Productividad (métrica del juego)
axes[3].plot(datos_juego['fecha'], datos_juego['productividad'], color='purple', linewidth=2)
axes[3].set_title('Índice de Productividad (Métrica del Juego)', fontsize=14, fontweight='bold')
axes[3].set_ylabel('Índice (0-100)')
axes[3].set_xlabel('Fecha')
axes[3].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('cusco_analisis_agricola.png', dpi=300, bbox_inches='tight')
print("📊 Gráfico guardado en: cusco_analisis_agricola.png")
plt.show()

# ============================================
# PASO 10: ESTADÍSTICAS PARA DOCUMENTACIÓN
# ============================================

print("\n" + "="*60)
print("📈 ESTADÍSTICAS PARA TU HACKATHON")
print("="*60)

print(f"\n📅 Período analizado: {datos_juego['fecha'].min()} a {datos_juego['fecha'].max()}")
print(f"📊 Total de observaciones: {len(datos_juego)}")

print("\n🌱 NDVI (Salud de Cultivos):")
print(f"   • Promedio: {datos_juego['NDVI'].mean():.3f}")
print(f"   • Máximo: {datos_juego['NDVI'].max():.3f}")
print(f"   • Mínimo: {datos_juego['NDVI'].min():.3f}")

print("\n🌡️ Temperatura:")
print(f"   • Promedio: {datos_juego['temperatura_celsius'].mean():.1f}°C")
print(f"   • Máxima: {datos_juego['temperatura_celsius'].max():.1f}°C")
print(f"   • Mínima: {datos_juego['temperatura_celsius'].min():.1f}°C")

print("\n💧 Precipitación:")
print(f"   • Total anual: {datos_juego['precipitacion_mm'].sum():.1f} mm")
print(f"   • Promedio diario: {datos_juego['precipitacion_mm'].mean():.1f} mm")
print(f"   • Día más lluvioso: {datos_juego['precipitacion_mm'].max():.1f} mm")

print("\n🎮 Métricas del Juego:")
print(f"   • Productividad promedio: {datos_juego['productividad'].mean():.1f}/100")
print(f"   • Días con estrés extremo: {(datos_juego['estres_calor'] == 'Extremo').sum()}")
print(f"   • Días con riego crítico: {(datos_juego['necesidad_riego'] == 'Crítico').sum()}")

print("\n✅ ¡Datos listos para tu videojuego!")
print("="*60)

# 🌍 Datos Satelitales Disponibles en Google Earth Engine (GEE)

En este proyecto usaremos diferentes variables satelitales que afectan la agricultura y pueden ser integradas al videojuego.

| Indicador / Variable            | Dataset en GEE                                   | Resolución | Qué mide (unidad) | Uso en Agricultura / Juego |
|---------------------------------|--------------------------------------------------|------------|-------------------|-----------------------------|
| **NDVI** (vegetación)           | `COPERNICUS/S2_SR_HARMONIZED` (Sentinel-2)       | 10 m       | Índice de vegetación (adimensional, -1 a 1) | Salud del cultivo, estrés |
| **EVI** (vegetación mejorada)   | `MODIS/061/MOD13Q1`                              | 250 m      | Índice de vegetación ajustado (adimensional, -1 a 1) | Más robusto en zonas densas |
| **Anomalías NDVI históricas**   | MODIS/Sentinel (comparando con medias históricas) | 250 m–10 m | Diferencia NDVI actual - promedio histórico | Detectar bajas campañas |
| **Temperatura superficial**     | `MODIS/061/MOD11A2`                              | 1 km       | Temperatura de la superficie (°C) | Estrés térmico, olas de calor |
| **Temperatura mínima / máxima** | `ECMWF/ERA5_LAND/HOURLY`                         | 9 km       | Temperatura del aire 2m (°C) | Heladas y calor extremo |
| **Precipitación diaria**        | `UCSB-CHG/CHIRPS/DAILY`                          | 5 km       | Lluvia acumulada diaria (mm/día) | Estimar agua disponible |
| **Anomalías de precipitación**  | CHIRPS histórico (comparando series temporales)  | 5 km       | Diferencia lluvia actual - promedio histórico (mm) | Fenómeno El Niño / La Niña |
| **Humedad del suelo**           | `NASA_USDA/HSL/SMAP10KM_soil_moisture`           | 10 km      | Humedad volumétrica del suelo (m³/m³) | Disponibilidad real de agua |
| **Evapotranspiración (ET)**     | `MODIS/061/MOD16A2`                              | 500 m      | Agua transferida suelo–atmósfera (mm/8 días) | Demanda hídrica del cultivo |
| **Índice de sequía (NDWI, ESI)**| MODIS/Sentinel                                   | 10–500 m   | Balance agua en vegetación (adimensional) | Estrés hídrico en plantas |
| **Biomasa (LAI, FAPAR)**        | `MODIS/061/MCD15A3H` (LAI/FAPAR)                 | 500 m      | LAI = área foliar (m²/m²), FAPAR = fracción de radiación absorbida (%) | Productividad fotosintética |
| **Cobertura terrestre**         | `MODIS/061/MCD12Q1`                              | 500 m      | Clasificación de uso del suelo (categorías) | Identificar uso agrícola del suelo |
| **Incendios / focos de calor**  | `FIRMS` (`FIRMS`)                                | 375 m      | Radiación térmica de incendios (MW) | Impacto en parcelas agrícolas |
| **Nubes persistentes**          | `COPERNICUS/S2_CLOUD_PROBABILITY`                | 10 m       | Probabilidad de nubosidad (%) | Interferencia en fotosíntesis |
| **Aerosoles / Contaminación**   | `COPERNICUS/S5P/OFFL/L3_AER_AI`                  | 1 km       | Profundidad óptica de aerosoles (adimensional) | Estrés ambiental en cultivos |

---

📌 **Nota sobre El Niño (ENOS):**  
GEE no incluye directamente el índice ONI (temperatura del Pacífico central), pero sí podemos usar **anomalías de precipitación (CHIRPS)** y **temperaturas ERA5** como indicadores indirectos.


# 🌾 Extracción de Datos Agrícolas para FarmGuardians
## Región: Global

Este notebook extrae datos satelitales de:
- 🌱 NDVI (Índice de Vegetación)
- 🌡️ Temperatura su,perficial
- 💧 Precipitación

**Fuente de datos:** Google Earth Engine (Sentinel-2, MODIS, CHIRPS)

---

## Paso 1: Clonar repositorio e instalar dependencias

En cada sesión de Colab debemos:
1. Clonar el repositorio desde GitHub
2. Instalar las librerías necesarias desde `requirements.txt`

In [ ]:
# Clonar el repositorio desde GitHub
!git clone https://github.com/iAmMazapan/FarmGuardians.git

# Instalar las dependencias desde requirements.txt
!pip install -r FarmGuardians/DataIntegrator/requirements.txt


fatal: destination path 'FarmGuardians' already exists and is not an empty directory.


---

## Paso 2: Configurar accesos

En este paso vamos a:
1. **Montar Google Drive** - Para guardar los resultados
2. **Autenticar Google Earth Engine** - Para acceder a datos satelitales

### ¿Por qué necesitamos esto?

- **Google Drive**: Los datos extraídos (CSV, JSON) se guardarán aquí para no perderlos
- **Google Earth Engine**: Es la plataforma de Google que nos da acceso gratuito a petabytes de imágenes satelitales

### Notas importantes:
- Te pedirá permisos para acceder a tu Drive
- La autenticación de GEE es mediante una cuenta de Google
- Solo necesitas hacerlo una vez por sesión

In [ ]:
# 1. Montar Google Drive
from google.colab import drive
drive.mount('/content/drive/')

print("✅ Google Drive montado en /content/drive/")
print("-" * 50)

# 2. Autenticar Google Earth Engine
import ee

print("\n Iniciando autenticación de Google Earth Engine...")
print("Se abrirá una ventana para autenticarte con tu cuenta de Google")

# Autenticar (se abre ventana emergente)
ee.Authenticate()

# Inicializar (sin proyecto específico por ahora)
ee.Initialize(project='farmguardians-gee')

print(" Google Earth Engine autenticado correctamente")
print("-" * 50)
print("\n🎉 Todo listo para extraer datos satelitales!")

Mounted at /content/drive/
✅ Google Drive montado en /content/drive/
--------------------------------------------------

 Iniciando autenticación de Google Earth Engine...
Se abrirá una ventana para autenticarte con tu cuenta de Google
 Google Earth Engine autenticado correctamente
--------------------------------------------------

🎉 Todo listo para extraer datos satelitales!


---

## Paso 3: Definir área de estudio en Cusco

Vamos a definir las coordenadas de nuestra zona de interés en Cusco, Perú.

### Opciones disponibles:

1. **Punto específico** - Para analizar una parcela o campo concreto
2. **Región amplia** - Para analizar toda la zona de Cusco
3. **Valle Sagrado** - Zona agrícola importante

### ¿Qué vamos a usar?

Para este análisis inicial usaremos **un punto específico cerca de la ciudad de Cusco** que tiene actividad agrícola visible desde satélite.

**Coordenadas seleccionadas:**
- Latitud: -13.5319°
- Longitud: -71.9675°
- Ubicación: Zona agrícola cerca de Cusco

Puedes cambiar estas coordenadas después si quieres analizar otra zona específica.

In [ ]:
import ee
import geemap
from ipywidgets import HBox

# Definir diferentes opciones de área de estudio

# Opción 1: Punto específico (zona agrícola cerca de Cusco)
cusco_point = ee.Geometry.Point([-71.9675, -13.5319])

# Opción 2: Región amplia de Cusco (departamento)
cusco_region = ee.Geometry.Polygon([[
    [-72.5, -13.0],
    [-71.0, -13.0],
    [-71.0, -14.5],
    [-72.5, -14.5],
    [-72.5, -13.0]
]])

# Opción 3: Valle Sagrado (importante zona agrícola)
valle_sagrado = ee.Geometry.Polygon([[
    [-72.2, -13.2],
    [-71.8, -13.2],
    [-71.8, -13.4],
    [-72.2, -13.4],
    [-72.2, -13.2]
]])

# Seleccionar geometría para análisis
area_estudio = valle_sagrado  # <-- Cambia esto si quieres usar otra opción

print("✅ Área de estudio definida")
print(f"📍 Tipo: {type(area_estudio).__name__}")
print(f"📍 Coordenadas: {area_estudio.getInfo()['coordinates']}")

# Crear mapa interactivo para visualizar
mapa_cusco = geemap.Map(center=[-13.5319, -71.9675], zoom=12)
mapa_cusco.addLayer(area_estudio, {'color': 'red'}, 'Área de Estudio')
mapa_cusco.add_basemap('HYBRID')  # Mapa satelital + etiquetas

print("\n🗺️ Visualizando mapa interactivo...")
display(mapa_cusco)

✅ Área de estudio definida
📍 Tipo: Geometry
📍 Coordenadas: [[[-72.2, -13.4], [-71.8, -13.4], [-71.8, -13.2], [-72.2, -13.2], [-72.2, -13.4]]]

🗺️ Visualizando mapa interactivo...


Map(center=[-13.5319, -71.9675], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Search…

---

## Paso 4: Definir funciones de extracción de datos

En este paso creamos las funciones que nos permitirán extraer datos de Google Earth Engine.

### Funciones que vamos a crear:

1. **`extraer_ndvi_sentinel2()`** - Extrae NDVI de Sentinel-2 (alta resolución: 10m)
2. **`extraer_ndvi_modis()`** - Extrae NDVI de MODIS (menor resolución: 250m, más histórico)
3. **`extraer_temperatura()`** - Extrae temperatura superficial en °Celsius
4. **`extraer_precipitacion()`** - Extrae precipitación diaria en milímetros
5. **`collection_a_dataframe()`** - Convierte los datos a formato tabla (pandas DataFrame)

### ¿Por qué tantas funciones?

Cada satélite tiene sus propias características y necesita un procesamiento específico:
- **Sentinel-2**: Necesita calcular NDVI y eliminar nubes
- **MODIS**: Ya tiene NDVI calculado pero necesita escalarse
- **Temperatura**: Necesita convertirse de Kelvin a Celsius
- **Precipitación**: Está lista para usar directamente

Estas funciones las usaremos en los siguientes pasos.

---

## Paso 5: Extraer datos satelitales (2023-2024)

Ahora vamos a extraer los datos reales usando las funciones que definimos.

### Período de análisis:
- **Inicio:** 1 de enero de 2023
- **Fin:** 31 de diciembre de 2024
- **Duración:** 2 años completos

### Datos que se extraerán:

| Dato | Satélite | Resolución | Frecuencia |
|------|----------|------------|------------|
| NDVI | Sentinel-2 | 10m | ~5 días |
| NDVI | MODIS | 250m | 16 días |
| Temperatura | MODIS | 1km | 8 días |
| Precipitación | CHIRPS | 5km | Diaria |

### ⏱️ Tiempo estimado:
Este proceso puede tardar **3-5 minutos** porque está descargando datos desde los servidores de Google Earth Engine.

### ⚠️ Nota importante:
Si ves warnings sobre "pixels", es normal. No te preocupes.